In [33]:
# 📌 외부 py 파일 수정 시 자동으로 재로드되도록 설정
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
# ================================================
# 📦 공통 패키지 import
# ================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import os
import sys

# src 경로 추가
PROJECT_ROOT = os.path.abspath(os.path.join(".."))
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# seaborn 스타일 설정
sns.set(style="whitegrid")

# ================================================
# ⚙️ 경로 및 상수 설정
# ================================================
DATA_DIR = "../data/processed"
OUTPUT_DIR = "../data/modeling"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# KMeans 설정
KMEANS_MODE = "manual"
KMEANS_MANUAL_K = 5

# XGBoost 설정
XGB_N_ESTIMATORS = 100

# MCLP 설정
COVERAGE_RADIUS = 0.02
FACILITY_LIMIT = 30

# ================================================
# 🧪 평가 함수 import
# ================================================
from evaluation.baseline_evaluator import (
    evaluate_existing_stations,
    evaluate_random_installation,
    evaluate_cluster_centers,
    evaluate_mclp_result
)


# K means

In [35]:
# 데이터 로드
grid = pd.read_csv(f"{DATA_DIR}/grid_system_processed.csv")

# 수요가 가장 높은 클러스터만 추출
grid = run_kmeans(
    grid,
    mode=KMEANS_MODE,
    manual_k=KMEANS_MANUAL_K,
    return_top_cluster_only=True
)

[MANUAL MODE] 수동 설정 k = 5

[Cluster별 평균 수요]
cluster
2    590.772472
3    584.367958
1    423.962670
4    381.162983
0     68.100964
Name: demand_score, dtype: float64

[필터링] 수요가 가장 높은 클러스터 (cluster=2)만 반환됨.


In [ ]:
# grid_features.csv 로드
features_all = pd.read_csv(f"{DATA_DIR}/grid_features.csv")

# KMeans로 추출한 수요 밀집 클러스터와 병합 (grid_id 기준)
features = features_all.merge(grid[['grid_id', 'cluster']], on='grid_id', how='inner')

# 확인
print(f"전체 feature 수: {len(features_all)}")
print(f"수요 밀집 클러스터 feature 수: {len(features)}")
print(f"사용 가능한 feature 컬럼: {features.columns.tolist()}")

# XGBoost

In [ ]:
from modeling.xgboost_model import train_and_predict

selected_features = ['supply_score', 'station_count', 'commercial_count', 'cluster']

features, metrics = train_and_predict(
    features,
    features=selected_features,
    n_estimators=XGB_N_ESTIMATORS
)

features.to_csv(f"{DATA_DIR}/grid_features_with_prediction.csv", index=False)
print("XGBoost 예측 성능:", metrics)

# MCLP

In [ ]:
from modeling.mclp_model import solve_mclp

# MCLP 실행
features = solve_mclp(
    df=features,
    coverage_radius=COVERAGE_RADIUS,
    facility_limit=FACILITY_LIMIT,
    demand_column='predicted_demand_score'
)

# 결과 저장
features.to_csv(f"{OUTPUT_DIR}/mclp_selected_sites.csv", index=False)

# 설치 위치 확인
selected = features[features['selected'] == 1]
print(f"설치 대상지 수: {len(selected)}")
print(f"커버 수요 총합: {selected['predicted_demand_score'].sum():,.2f}")

# 평가

In [ ]:
# features 불러오기 (예측 수요 포함된 grid 데이터)
features = pd.read_csv(f"{DATA_DIR}/grid_features_with_prediction.csv")

# 기존 충전소 데이터 로드
stations = pd.read_csv(f"{DATA_DIR}/raw/한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20230531.csv", encoding='cp949')

# 평가 실행
baseline1 = evaluate_existing_stations(features, stations)
baseline2 = evaluate_random_installation(features, n=FACILITY_LIMIT)
baseline3 = evaluate_cluster_centers(features)
baseline4 = evaluate_mclp_result(features)

In [ ]:
results = pd.DataFrame([
    {'baseline': '① 기존 충전소', **baseline1},
    {'baseline': '② 랜덤 설치', **baseline2},
    {'baseline': '③ 클러스터 중심', **baseline3},
    {'baseline': '④ MCLP 최적화', **baseline4}
])

from IPython.display import display
display(results)